<img src="https://www.colorado.edu/rc/sites/default/files/page/logo.png"
     alt="Logo for Research Computing @ University of Colorado Boulder"
     width="400" />
     
# Overview over `ipyparallel`

## Introduction

`IPython` supports many different styles of parallelism:
- Task parallelism
- Data parallel
- Single program, multiple data parallelism
- Multiple program, multiple data parallelism
- Combination of all approaches above


## Typical use cases

- Quickly parallelize algorithms that are embarrassingly parallel.
- Analyze and visualize large datasets interactively using matplotlib or other python libraries.
- Run a set of tasks on several nodes of a cluster using dynamic load balancing.
- Steer a MPI based simulation on a supercomputer from on IPython session on your laptop
- Develop, test, and debug parallel algorithms that use MPI interactively


## Architecture overview

When we start an `ipcluster` you generally start one controller process and several engines. The controller and engines do not necessarily run on the same node (or computer).

Most of the code will probably run on the controller, but you will run the compute intensive portions of your program on the engines.

The engines need to receive data and code from the controller:

<img src="./controller_send.png"
     alt="Controller with 2 engines sending data and code"
     width="300" />
    <img src="./controller_receive.png"
     alt="Controller with 2 engines receiving data"
     width="300" /> 

# Easy parallelism using `map`

## Starting an `ipcluster`

Using the cluster tab or

In [ ]:
!ipcluster start -n 4 --daemonize

## Loading the right modules

In [ ]:
import ipyparallel

## Create a client instance, used to connect the controller to the remote engines

In [ ]:
profile = 'example-shas'
rc = ipyparallel.Client(profile=profile)
nengines = len(rc)
nengines

In [ ]:
rc.ids

## Create a `DirectView`

A `DirectView` is created by list access to a client instance:

In [ ]:
dvall  = rc[:]
print(dvall)

In [ ]:
type(dvall)

## By default only the controller executes code

In [ ]:
import socket
print('\n Running on',socket.gethostname())

## Parallelizing the serial image example
Converting a list of images to grayscale

In [ ]:
def convert2greyscale(path):
    from skimage.io import imread
    from skimage.color import rgb2gray
    img = imread(path)
    img_gray = rgb2gray(img)
    return img_gray

Note: importing the modules inside the functions avoids problems accessing modules on the engines. 
Other approaches are possible and will be discussed in later notebooks.

In [ ]:
import os
pictures_dir = os.path.join('.', 'images', 'cornflower')
pictures = []
for directory, subdirs, files in os.walk(pictures_dir):
    for fname in files:
        if fname.lower().endswith(('.jpg', '.png')):
            pictures.append(os.path.join(directory, fname))

### The serial exampel for reference

In [ ]:
%cd /home/thha0714/Projects/RMACC-2019-ipyparallel

In [ ]:
%%time
sconverted = [*map(convert2greyscale, pictures[:32])]

### Parallelizing using the parallel map function 

In [ ]:
%%time
pconverted = dvall.map_sync(convert2greyscale, pictures[:32])

   ### Try again with a load-balanced view

In [ ]:
dlb = rc.load_balanced_view()

In [ ]:
%%time
lbconverted = dlb.map_sync(convert2greyscale, pictures[:32])